In [ ]:
####### Run a simple analysis of terrain around a selected location

# Step 1: Sign up for a Google Earth Engine Account here:
# https://earthengine.google.com/signup/

# you will use this to authenticate your account with GEE to run the script and
# utilize GEE layers

In [ ]:
# get any missing modules
! pip install earthengine-api geemap

In [1]:
# setup modules
import numpy as np
import matplotlib.pyplot as pl
import matplotlib as mpl
import sys,os,os.path,tempfile

# spatial modules
import ee
#import geemap # currently not using as colorbars dont load properly in google collab
import geemap.colormaps as cm
import geemap.foliumap as geemap # use folium instead so coloramps work on goolge collab

# disable after updating
#geemap.update_package()

In [2]:
## setup output. defined dir or system temp dir
data_dir=os.path.expanduser(os.path.join(tempfile.gettempdir(),'uas_output'))

if not os.path.exists(data_dir):
    os.makedirs(data_dir)

print(" ")
print("****** Raster and blockage results output folder ******")
print("****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******")
print("****** Use the dir navigation to go up one level, then navigate to tmp/geo ******")
print(data_dir)

 
****** Raster and blockage results output folder ******
****** Note: In Google Collab, use the dir navigation tool on the left of the screen ******
****** Use the dir navigation to go up one level, then navigate to tmp/geo ******
/var/folders/xp/h3k9vf3n2jx181ts786_yjrn9c2gjq/T/uas_output


In [3]:
##### initialize ee API  - you need to authenticate with GEE and initialize

## Trigger the authentication flow. You only need to do this once while running notebook
ee.Authenticate()

## After inserting the API key initialize GEE
ee.Initialize()

In [4]:
### get cogs file
# https://blog.gishub.org/gee-tutorial-38-how-to-use-cloud-optimized-geotiff-with-earth-engine
URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_RGB_cog.tif'
kg_rgb_image = geemap.load_GeoTIFF(URL)

RGB_imageVisParam = {"opacity":1,"bands":["B0","B1","B2"],
                    "min":-4.063213446967957,"max":85.22120105681381,
                     "gamma":1};

URL = 'gs://bnl_uas_data/NGEEArctic_UAS_Kougarok_20180725_Flight6_CHM_cog.tif'
kg_chm_image = geemap.load_GeoTIFF(URL)

dem_palette = cm.get_palette('Spectral_r', n_class=30)
demViz = {'min': 0.0, 'max': 150.0, 'palette': dem_palette, 'opacity': 1}

In [5]:
# center map over Kougarok
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);

uas_map.addLayer(kg_rgb_image, RGB_imageVisParam, 'UAS_Kougarok_20180725_Flight6_RGB')
uas_map

In [6]:
# center map over Kougarok
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);

chm_mask = kg_chm_image.gt(0);
chm_uas_image_masked = kg_chm_image.updateMask(chm_mask);

uas_map.addLayer(chm_uas_image_masked, demViz, 'UAS_Kougarok_20180725_Flight6_CHM')

uas_map

In [7]:
### Combined map

# center map over Kougarok
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=18);
uas_map.addLayer(kg_rgb_image, RGB_imageVisParam, 'UAS_Kougarok_20180725_Flight6_RGB')
demViz = {'min': 0.0, 'max': 150.0, 'palette': dem_palette, 'opacity': 0.5}
uas_map.addLayer(chm_uas_image_masked, demViz, 'UAS_Kougarok_20180725_Flight6_CHM')


uas_map



In [8]:
### Extract random points through CHM layer
uas_map = geemap.Map(center=[65.164354,-164.823043], zoom=17);
demViz = {'min': 0.0, 'max': 150.0, 'palette': dem_palette, 'opacity': 1}
chm_mask = kg_chm_image.gt(0);
chm_uas_image_masked = kg_chm_image.updateMask(chm_mask);

uas_map.addLayer(chm_uas_image_masked, demViz, 'UAS_Kougarok_20180725_Flight6_CHM')

region = ee.Geometry.Point([-164.8239, 65.1644]).buffer(30)
uas_map.addLayer(ee.Image().paint(region, 0, 4), {'palette': 'yellow'}, "ROI-1")

random_points = geemap.random_sampling(chm_uas_image_masked, region, numPixels=100)
uas_map.addLayer(random_points, {}, "Random Sampling")

uas_map

In [9]:
### Show results of random sampling (x100)
df = geemap.ee_to_pandas(random_points)
df

,B0
0,84.078217
1,7.678223
2,5.362701
3,37.515259
4,22.560120
...,...
95,25.104523
96,0.373840
97,39.096832
98,119.136047
